In [1]:
import numpy as np
import torch
import json

from torch.utils.data import DataLoader

from collections import defaultdict

from GAIL.models.nets import Expert
from GAIL.models.gail import GAIL

from utilities import *
from utilitiesDL import *

In [3]:
DataFGMDir = '/project/iarpa/wifiHAR/HAR_survey/window_FGM/'
LSTMModelDir = './savedModels/selected/'

GAILModelJsonFileName = './GAIL/GAILModelConfig.json'
GAILModelJson = json.load(open(GAILModelJsonFileName))

GAILJsonFileName = './inputJson/GAIL/test.json'
GAILJson = json.load(open(GAILJsonFileName))

LSTMModelName = GAILJson['LSTMModelName']
noiseAmpRatio = GAILJson['noiseAmpRatio']
trDataRatio = GAILJson['trDataRatio']
cudaID = GAILJson['cudaID']

torch.set_num_threads(1)

if cudaID >= 0:
    device = torch.device("cuda:"+str(cudaID))
    cudaAvbl = True
else:
    device = torch.device("cpu")
    cudaAvbl = False

dataType = LSTMModelName.split('_')[0]

if dataType == 'survey':
    fs = 1000 # 1 kHz
    nSubC = 30
    nRX = 3
    
    winLen = 1000
    thres = 60
    slideLen = 400
    activities = ['fall', 'pickup', 'run', 'sitdown', 'standup', 'walk']

LSTMType = LSTMModelName.split('_')[1]
bidirectional = (LSTMType == 'BLSTM')
nHidden = int(LSTMModelName.split('_')[3])
threshold = int(LSTMModelName.split('_')[5])
nLayer = int(LSTMModelName.split('_')[7])

# Load the LSTM model
HARNet = LSTMNet(nClasses=len(activities), input_size=nSubC*nRX, bidirectional=bidirectional,\
                hidden_size=nHidden, num_layers=1, seq_length=winLen//2, device=device)
HARNet.load_state_dict(torch.load(LSTMModelDir + LSTMModelName + '.cpkt'))
HARNet.to(device)

# Load dataset labelled with FGM attack
FGMdatasetDir = '/project/iarpa/wifiHAR/HAR_' + dataType + '/window_FGM/'
dataDict = {file:[] for file in activities}
tsDataDict = {file:[] for file in activities}

trDataset = list()
tsDataset = list()
for actInd, activity in enumerate(activities):
    dataDict[activity] = defaultdict(list)

    dataActFileName = FGMdatasetDir + LSTMModelName + '_' + activity + '.pt'
    dataAct = torch.load(dataActFileName)

    dataDict[activity]['obs'] =\
        torch.reshape(torch.squeeze(dataAct[0, :, :]), (-1, winLen//2, nSubC*nRX)).detach().cpu().numpy()
    dataDict[activity]['FGM'] = noiseAmpRatio *\
        torch.reshape(torch.squeeze(torch.squeeze(dataAct[1, :, :])), (-1, winLen//2, nSubC*nRX)).detach().cpu().numpy()
    dataDict[activity]['label'] =\
        actInd * np.ones((dataDict[activity]['obs'].shape[0]), dtype=int)

    datasetAct = FGMDataset(dataDict[activity], device)

    trDataset.append(torch.utils.data.Subset(datasetAct, range(int(0.8*len(datasetAct)))))
    tsDataset.append(torch.utils.data.Subset(datasetAct, range(int(0.8*len(datasetAct)), len(datasetAct))))

    print('activity:', activity, 'trDataset:', len(trDataset[-1]), 'tsDataset:', len(tsDataset[-1]))
    
trLoader = DataLoader(torch.utils.data.ConcatDataset(trDataset),\
                      batch_size=2, shuffle=True, generator=torch.Generator(device='cuda'))
tsLoader = DataLoader(torch.utils.data.ConcatDataset(tsDataset),\
                      batch_size=10, shuffle=True, generator=torch.Generator(device='cuda'))

print('trLoader:', len(trLoader), 'tsLoader:', len(tsLoader))

activity: fall trDataset: 354 tsDataset: 89
activity: pickup trDataset: 396 tsDataset: 99
activity: run trDataset: 967 tsDataset: 242
activity: sitdown trDataset: 329 tsDataset: 83
activity: standup trDataset: 244 tsDataset: 61
activity: walk trDataset: 1172 tsDataset: 293
trLoader: 1731 tsLoader: 87


In [4]:
model = GAIL(state_dim=nSubC*nRX, action_dim=nSubC*nRX,\
             discrete=False, train_config=GAILModelJson).to(device)

noiseAmpRatioList = [1e-5, 1e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 7.5e-2, 0.1, 0.2, 0.5]
for noiseAmpRatio in noiseAmpRatioList:
    accTest = getAcc(tsLoader, HARNet, 'FGM', noiseAmpRatio, noiseType='FGM')
    print('noiseAmpRatio:', noiseAmpRatio, 'accTest:', accTest)

# model.train(HARNet, trLoader, tsLoader)

noiseAmpRatio: 1e-05 accTest: 0.9400230680507498
noiseAmpRatio: 0.0001 accTest: 0.9331026528258363
noiseAmpRatio: 0.0005 accTest: 0.9077277970011534
noiseAmpRatio: 0.001 accTest: 0.881199538638985
noiseAmpRatio: 0.002 accTest: 0.8373702422145328
noiseAmpRatio: 0.005 accTest: 0.6782006920415224
noiseAmpRatio: 0.01 accTest: 0.4809688581314879
noiseAmpRatio: 0.02 accTest: 0.3102652825836217
noiseAmpRatio: 0.05 accTest: 0.2041522491349481
noiseAmpRatio: 0.075 accTest: 0.18569780853517878
noiseAmpRatio: 0.1 accTest: 0.18223760092272204
noiseAmpRatio: 0.2 accTest: 0.14071510957324107
noiseAmpRatio: 0.5 accTest: 0.1061130334486736


In [37]:
import numpy as np
import torch

from torch.nn import Module

from GAIL.models.nets import PolicyNetwork, ValueNetwork, Discriminator
from GAIL.utils.funcs import get_flat_grads, get_flat_params, set_params, \
    conjugate_gradient, rescale_and_linesearch

if torch.cuda.is_available():
    from torch.cuda import FloatTensor
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
else:
    from torch import FloatTensor


class GAIL(Module):
    def __init__(
        self,
        state_dim,
        action_dim,
        discrete,
        train_config=None
    ) -> None:
        super().__init__()

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.discrete = discrete
        self.train_config = train_config

        self.pi = PolicyNetwork(self.state_dim, self.action_dim, self.discrete)
        self.v = ValueNetwork(self.state_dim)

        self.d = Discriminator(self.state_dim, self.action_dim, self.discrete)

    def get_networks(self):
        return [self.pi, self.v]

    def act(self, state):
        self.pi.eval()

        state = FloatTensor(state)
        distb = self.pi(state)

        action = distb.sample().detach().cpu().numpy()

        return action

    def train(self, HARNet, trLoader, tsLoader, render=False):
        num_iters = self.train_config["num_iters"]
        num_steps_per_iter = self.train_config["num_steps_per_iter"]
        horizon = self.train_config["horizon"]
        lambda_ = self.train_config["lambda"]
        gae_gamma = self.train_config["gae_gamma"]
        gae_lambda = self.train_config["gae_lambda"]
        eps = self.train_config["epsilon"]
        max_kl = self.train_config["max_kl"]
        cg_damping = self.train_config["cg_damping"]
        normalize_advantage = self.train_config["normalize_advantage"]

        opt_d = torch.optim.Adam(self.d.parameters())

        exp_acc_iter = []

        exp_obs = []
        exp_acts = []

        # nTsIter = len(tsLoader)
        # print(nTsIter)
        
        noiseAmpRatioList = [1e-5, 1e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 7.5e-2, 0.1, 0.2, 0.5]
        for noiseAmpRatio in noiseAmpRatioList:
            accTest = getAcc(tsLoader, HARNet, noiseAmpRatio, noiseType='FGM')
            print('noiseAmpRatio:', noiseAmpRatio, 'accTest:', accTest)
        
        # for tsData in tsLoader:
        #     print(tsData['input'].shape, tsData['label'].shape)
        #     exp_obs.append(tsData['input'])
        #     exp_acts.append(tsData['label'])            
            # ep_rwds = FloatTensor(ep_rwds)

        # exp_rwd_mean = np.mean(exp_rwd_iter)
        # print(
        #     "Expert Reward Mean: {}".format(exp_rwd_mean)
        # )

        # print(exp_obs.shape, exp_acts.shape)
        # exp_obs = FloatTensor(np.array(exp_obs))
        # exp_acts = FloatTensor(np.array(exp_acts))
        # return exp_rwd_mean, rwd_iter_means
